## STEP 1


In [6]:
import tarfile

def dump_tar_as_hex(tar_file_path):
    with tarfile.open(tar_file_path, 'r') as tar:
        for member in tar.getmembers():
        
            with tar.extractfile(member) as file:
                data = file.read()
                offset = 0
                while data:
                    chunk = data[:16]
                    hex_str = ' '.join(f'{byte:04x}' for byte in chunk)
                    text_str = ''.join(chr(byte) if 32 <= byte <= 126 else '.' for byte in chunk)
                    print(f'{offset:08x}: {hex_str.ljust(48)} {text_str}')
                    data = data[16:]
                    offset += 16
            print('\n')


dump_tar_as_hex('files.tar')


00000000: 0046 0069 006c 0065 0020 0031 0020 0063 006f 006e 0074 0065 006e 0074 0073 000a File 1 contents.


00000000: 0046 0069 006c 0065 0020 0032 0020 0063 006f 006e 0074 0065 006e 0074 0073 000a File 2 contents.


00000000: 0046 0069 006c 0065 0020 0033 0020 0063 006f 006e 0074 0065 006e 0074 0073 000a File 3 contents.




## STEP 2 ( little endian just means reverse ordering)

In [26]:
def dump_tar_as_hex(tar_file_path, little_endian=False, group_size=2):
    with tarfile.open(tar_file_path, 'r') as tar:
        for member in tar.getmembers():
            
            with tar.extractfile(member) as file:
                data = file.read()
                offset = 0
                while data:
                    chunk = data[:group_size * 2]
                    hex_str = ' '.join(f'{byte:04x}' for byte in chunk)
                    text_str = ''.join(chr(byte) if 32 <= byte <= 126 else '.' for byte in chunk)
                    if little_endian:
                        chunk = chunk[::-1]
                    print(f'{offset:08x}: {hex_str.ljust(48)} {text_str}')
                    data = data[group_size * 2:]
                    offset += group_size * 2
            

# Example usage:
dump_tar_as_hex('files.tar', group_size=2)




00000000: 0046 0069 006c 0065                              File
00000004: 0020 0031 0020 0063                               1 c
00000008: 006f 006e 0074 0065                              onte
0000000c: 006e 0074 0073 000a                              nts.
00000000: 0046 0069 006c 0065                              File
00000004: 0020 0032 0020 0063                               2 c
00000008: 006f 006e 0074 0065                              onte
0000000c: 006e 0074 0073 000a                              nts.
00000000: 0046 0069 006c 0065                              File
00000004: 0020 0033 0020 0063                               3 c
00000008: 006f 006e 0074 0065                              onte
0000000c: 006e 0074 0073 000a                              nts.


In [28]:
def dump_tar_as_hex_with_seek(tar_file_path, seek_offset=0, octets_per_line=16, columns_per_line=16):
    with tarfile.open(tar_file_path, 'r') as tar:
        for member in tar.getmembers():
            print(f"Dumping contents of {member.name} as hex (with seek):")
            with tar.extractfile(member) as file:
                file.seek(seek_offset)
                data = file.read()
                offset = seek_offset
                while data:
                    chunk = data[:octets_per_line]
                    hex_str = ' '.join(f'{byte:02x}' for byte in chunk)
                    text_str = ''.join(chr(byte) if 32 <= byte <= 126 else '.' for byte in chunk)
                    print(f'{offset:08x}: {hex_str.ljust(columns_per_line*3)} {text_str}')
                    data = data[octets_per_line:]
                    offset += octets_per_line
            print('\n')

# Example usage:
dump_tar_as_hex_with_seek('files.tar', seek_offset=0, octets_per_line=16, columns_per_line=4)


Dumping contents of file1.txt as hex (with seek):
00000000: 46 69 6c 65 20 31 20 63 6f 6e 74 65 6e 74 73 0a File 1 contents.


Dumping contents of file2.txt as hex (with seek):
00000000: 46 69 6c 65 20 32 20 63 6f 6e 74 65 6e 74 73 0a File 2 contents.


Dumping contents of file3.txt as hex (with seek):
00000000: 46 69 6c 65 20 33 20 63 6f 6e 74 65 6e 74 73 0a File 3 contents.


